In [ ]:
import json
import os
import pysolr
import requests
import pandas as pd

CORE_NAME = "IRF24P1T"
VM_IP = "localhost"

In [ ]:
def delete_core(core=CORE_NAME):
    print(os.system('sudo su - solr -c "/opt/solr/bin/solr delete -c {core}"'.format(core=core)))


def create_core(core=CORE_NAME):
    print(os.system(
        'sudo su - solr -c "/opt/solr/bin/solr create -c {core} -n data_driven_schema_configs"'.format(
            core=core)))

In [ ]:
class Indexer:
    def __init__(self):
        self.solr_url = f'http://{VM_IP}:8983/solr/'
        self.connection = pysolr.Solr(self.solr_url + CORE_NAME, always_commit=True, timeout=5000000)

    def do_initial_setup(self):
        delete_core()
        create_core()

    def create_documents(self, docs):
        print(self.connection.add(docs))

    def add_fields(self):
        data = {
            "add-field": [
                {
                    "name": "field_1",
                    "type": "string",
                    "indexed": True,
                    "multiValued": False
                },
                {
                    "name": "field_2",
                    "type": "string",
                    "multiValued": False
                },
            ]
        }

        print(requests.post(self.solr_url + CORE_NAME + "/schema", json=data).json())

In [ ]:
data = {
    'field_1': ['This contains sample text A', 'This contains sample text C'],
    'field_2': ['This contains sample text B', 'This contains sample text D']
    }
df = pd.DataFrame(data)
df

,field_1,field_2
0,This contains sample text A,This contains sample text B
1,This contains sample text C,This contains sample text D


In [ ]:
# Setting up the core and adding the fields
i = Indexer()
i.do_initial_setup()
i.add_fields()

256
0
{'responseHeader': {'status': 0, 'QTime': 298}}


In [ ]:
# Index the sample dataset
collection = df.to_dict('records')
i.create_documents(collection)

{
  "responseHeader":{
    "status":0,
    "QTime":67
  }
}
